In [1]:
import re
import joblib
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import re
import urllib.request
from konlpy.tag import Okt

In [9]:
train_df = pd.read_csv('../../../machine-Learning/00.data/naverMovie/train.tsv',
                 sep='\t')
#train_df = train_df.drop(['id'], axis=1, inplace=False)
train_df.head()

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [104]:
test_df= pd.read_csv('../../../machine-Learning/00.data/naverMovie/test.tsv',
                 sep='\t')
test_df.head()

,Unnamed: 0,id,document,label
0,0,6270596,굳 ㅋ,1
1,2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
2,3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
3,4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
4,5,7898805,음악이 주가 된 최고의 음악영화,1


형태소 분석>

In [10]:
okt = Okt()

In [12]:

stopwords = [ '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']  # 안에꺼 빠진상태로 돌림

In [13]:
from tqdm import tqdm_notebook
X_test = []
for sentence in tqdm_notebook(test_df['document']):
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = ' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
    X_test.append(temp_X)

In [14]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df['document']):    
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    
    temp_X =' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
    X_train.append(temp_X)

In [15]:
y_train = train_df.label.values
y_test = test_df.label.values

Case 1. CountVectorizer + LogisticRegression

In [35]:

cvecter = CountVectorizer()

In [36]:
cvecter.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [37]:
pipeline1 = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression())
])
params = ({
    'count_vect__ngram_range': [(1,1), (1,2)],
    'count_vect__max_df': [0.9, 0.95, 1],
    'lr_clf__C': [1, 3, 5, 10]
})

In [38]:
grid_pipe = GridSearchCV(pipeline1, param_grid=params, cv=3,
                         scoring='accuracy')
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

{'count_vect__max_df': 0.9, 'count_vect__ngram_range': (1, 2), 'lr_clf__C': 1} 0.8416157375969711


In [43]:
best_count_lr = grid_pipe.best_estimator_
pred_count_lr = best_count_lr.predict(X_test)
accuracy_score(y_test, pred_count_lr)

0.8463720787835494

In [44]:
joblib.dump(best_count_lr, '../static/model/Npipeline_cl.pkl')

['../static/model/Npipeline_cl.pkl']

Case 2. TfidfVectorizer + LogisticRegression

In [21]:
pipeline2 = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

params = {
    'tfidf_vect__ngram_range': [(1,1), (1,2)],
    'tfidf_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
}

In [23]:
%time pipeline2.fit(X_train, y_train)

Wall time: 28.1 s


Pipeline(steps=[('tfidf_vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('lr_clf', LogisticRegression(C=10))])

In [25]:
grid_pipe = GridSearchCV(pipeline2, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  4.0min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.8205307597862693


In [42]:
best_tfidf_lr = grid_pipe.best_estimator_
pred_tfidf_lr = best_tfidf_lr.predict(X_test)
accuracy_score(y_test, pred_tfidf_lr)

0.8463720787835494

In [46]:
joblib.dump(best_tfidf_lr, '../static/model/Npipeline_tl.pkl')

['../static/model/Npipeline_tl.pkl']

In [28]:
nb = MultinomialNB()

Case 3. CountVectorizer + MultinomialNB

In [60]:
pipeline3= Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('nv_clf', MultinomialNB())
])
params = {
    'count_vect__ngram_range': [ (1,2)],
    'count_vect__max_df': [0.9,0.99]}
  



In [61]:
%time pipeline3.fit(X_train, y_train)

Wall time: 8.61 s


Pipeline(steps=[('count_vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('nv_clf', MultinomialNB())])

In [62]:
grid_pipe = GridSearchCV(pipeline3, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   41.9s finished
{'count_vect__max_df': 0.9, 'count_vect__ngram_range': (1, 2)} 0.8422056231180252


In [63]:
best_count_cn = pipeline3
pred_count_cn = best_count_cn.predict(X_test)
accuracy_score(y_test, pred_count_cn)

0.8441881824675987

In [64]:
joblib.dump(best_count_cn, '../static/model/Npipeline_cn.pkl')

['../static/model/Npipeline_cn.pkl']

Case 4. TfidfVectorizer + MultinomialNB

In [65]:
pipeline4 = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('nv_clf', MultinomialNB())
])

params = {
    'tfidf_vect__ngram_range': [ (1,2)],
    'tfidf_vect__max_df': [0.9,0.99]}

In [68]:
%time pipeline4.fit(X_train, y_train)

Wall time: 9.25 s


Pipeline(steps=[('tfidf_vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('nv_clf', MultinomialNB())])

In [69]:
grid_pipe = GridSearchCV(pipeline4, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1) #, n_jobs=-1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   44.3s finished
{'tfidf_vect__max_df': 0.9, 'tfidf_vect__ngram_range': (1, 2)} 0.8433785350261677


In [70]:
best_tfidf_nv = pipeline4
pred_tfidf_nv = best_tfidf_nv.predict(X_test)
accuracy_score(y_test, pred_tfidf_nv)

0.8460251046025105

In [71]:
joblib.dump(best_tfidf_nv, '../static/model/Npipeline_tn.pkl')

['../static/model/Npipeline_tn.pkl']

테스트>

In [72]:
model_cl = joblib.load('../static/model/Npipeline_cl.pkl')
model_tl = joblib.load('../static/model/Npipeline_tl.pkl')
model_cn = joblib.load('../static/model/Npipeline_cn.pkl')
model_tn = joblib.load('../static/model/Npipeline_tn.pkl')

In [100]:
index = 2
test_data = []
test_data.append(test_df.iloc[index, 2])

In [101]:
pred_cl = model_cl.predict(test_data)
pred_cl[0]

1

In [103]:
pred_cl = model_cl.predict(test_data)
pred_cn = model_cn.predict(test_data)
pred_tl = model_tl.predict(test_data)
pred_tn = model_tn.predict(test_data)
pred_cl[0], pred_cn[0], pred_tl[0],  pred_tn[0]

(1, 1, 1, 1)

In [107]:
review2 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ '
review2= re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review2)
morphs =okt.morphs(review2)
review2 = ' '.join([word for word in morphs if not word in stopwords])
review2

'진짜 개 노잼 이다 편이 랑 같은 감독 맞나 러닝 타임 도 길어서 개 지루함 ㄹㅇ'

In [108]:
test_data = []
test_data.append(review2)

In [109]:
pred_cl = model_cl.predict(test_data)
pred_cn = model_cn.predict(test_data)
pred_tl = model_tl.predict(test_data)
pred_tn = model_tn.predict(test_data)
pred_cl[0], pred_cn[0], pred_tl[0] , pred_tn[0]

(0, 0, 0, 0)

In [75]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ '

In [76]:
review1= re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review1)

In [81]:
morphs =okt.morphs(review1)
review1 = ' '.join([word for word in morphs if not word in stopwords])
review1

'진짜 개 노잼 이다 편이 랑 같은 감독 맞나 러닝 타임 도 길어서 개 지루함 ㄹㅇ'

In [90]:
cvecter = CountVectorizer()
cvecter.fit(X_train)
X_train_cvect = cvecter.transform(X_train)
X_test_cvect = cvecter.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cvect, y_train)
pred = lr_clf.predict(X_test_cvect)
accuracy_score(y_test, pred)

0.8264312684967854

In [91]:
review_cvect = cvecter.transform([review1])

In [92]:
pred = lr_clf.predict(review_cvect)
pred[0]

0

In [97]:
test_data = []
test_data.append(review1)

In [99]:
pred_cl = model_cl.predict(test_data)
pred_cn = model_cn.predict(test_data)
pred_tl = model_tl.predict(test_data)
pred_tn = model_tn.predict(test_data)
pred_cl[0], pred_cn[0], pred_tl[0] , pred_tn[0]

(0, 0, 0, 0)

In [ ]:
#test_data.append(df.iloc[index,2]) # test_data.append(df.loc[index,'document']) 

In [106]:
test_df.label[2]

0